    We will do
    1. Fit a logistic regression model to only one predictor and understand how to interpret the results.
    2. Fit the logistic regression model to the entire dataset.
    3. Take the results from the logistic regression model to create a bootstrap with confidence intervals.

In [1]:
import sys
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)
pd.set_option('display.width', 1000)
pd.set_option('display.notebook_repr_html', True)
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.api import OLS
from statsmodels.api import add_constant
from statsmodels.regression.linear_model import RegressionResults
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from collections import Counter
sns.set(style="ticks")
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

matplotlib.rcParams['figure.figsize'] = (13.0, 6.0)


#### Loading Data

In [2]:
df = pd.read_csv('/cancer_data_log.csv')
df.head()

,Y,Gene 1,Gene 2,Gene 3,Gene 4,Gene 5,Gene 6,Gene 7,Gene 8,Gene 9,Gene 10,Gene 11,Gene 12,Gene 13,Gene 14,Gene 15,Gene 16,Gene 17,Gene 18,Gene 19,Gene 20,Gene 21,Gene 22,Gene 23,Gene 24,Gene 25,Gene 26,Gene 27,Gene 28,Gene 29,Gene 30,Gene 31,Gene 32,Gene 33,Gene 34,Gene 35,Gene 36,Gene 37,Gene 38,Gene 39,Gene 40,Gene 41,Gene 42,Gene 43,Gene 44,Gene 45,Gene 46,Gene 47,Gene 48,Gene 49,Gene 50,Gene 51,Gene 52,Gene 53,Gene 54,Gene 55,Gene 56,Gene 57,Gene 58,Gene 59,Gene 60,Gene 61,Gene 62,Gene 63,Gene 64,Gene 65,Gene 66,Gene 67,Gene 68,Gene 69,Gene 70,Gene 71,Gene 72,Gene 73,Gene 74,Gene 75,Gene 76,Gene 77,Gene 78,Gene 79,Gene 80,Gene 81,Gene 82,Gene 83,Gene 84,Gene 85,Gene 86,Gene 87,Gene 88,Gene 89,Gene 90,Gene 91,Gene 92,Gene 93,Gene 94,Gene 95,Gene 96,Gene 97,Gene 98,Gene 99,Gene 100,Gene 101,Gene 102,Gene 103,Gene 104,Gene 105,Gene 106,Gene 107,Gene 108,Gene 109,Gene 110,Gene 111,Gene 112,Gene 113,Gene 114,Gene 115,Gene 116,Gene 117,Gene 118,Gene 119,Gene 120,Gene 121,Gene 122,Gene 123,Gene 124,Gene 125,Gene 126,Gene 127,Gene 128,Gene 129,Gene 130,Gene 131,Gene 132,Gene 133,Gene 134,Gene 135,Gene 136,Gene 137,Gene 138,Gene 139,Gene 140,Gene 141,Gene 142,Gene 143,Gene 144,Gene 145,Gene 146,Gene 147,Gene 148,Gene 149,Gene 150,Gene 151,Gene 152,Gene 153,Gene 154,Gene 155,Gene 156,Gene 157,Gene 158,Gene 159,Gene 160,Gene 161,Gene 162,Gene 163,Gene 164,Gene 165,Gene 166,Gene 167,Gene 168,Gene 169,Gene 170,Gene 171,Gene 172,Gene 173,Gene 174,Gene 175,Gene 176,Gene 177,Gene 178,Gene 179,Gene 180,Gene 181,Gene 182,Gene 183,Gene 184,Gene 185,Gene 186,Gene 187,Gene 188,Gene 189,Gene 190,Gene 191,Gene 192,Gene 193,Gene 194,Gene 195,Gene 196,Gene 197,Gene 198,Gene 199,Gene 200,Gene 201,Gene 202,Gene 203,Gene 204,Gene 205,Gene 206,Gene 207,Gene 208,Gene 209,Gene 210,Gene 211,Gene 212,Gene 213,Gene 214,Gene 215,Gene 216,Gene 217,Gene 218,Gene 219,Gene 220,Gene 221,Gene 222,Gene 223,Gene 224,Gene 225,Gene 226,Gene 227,Gene 228,Gene 229,Gene 230,Gene 231,Gene 232,Gene 233,Gene 234,Gene 235,Gene 236,Gene 237,Gene 238,Gene 239,Gene 240,Gene 241,Gene 242,Gene 243,Gene 244,Gene 245,Gene 246,Gene 247,Gene 248,Gene 249,Gene 250,Gene 251,Gene 252,Gene 253,Gene 254,Gene 255,Gene 256,Gene 257,Gene 258,Gene 259,Gene 260,Gene 261,Gene 262,Gene 263,Gene 264,Gene 265,Gene 266,Gene 267,Gene 268,Gene 269,Gene 270,Gene 271,Gene 272,Gene 273,Gene 274,Gene 275,Gene 276,Gene 277,Gene 278,Gene 279,Gene 280,Gene 281,Gene 282,Gene 283,Gene 284,Gene 285,Gene 286,Gene 287,Gene 288,Gene 289,Gene 290,Gene 291,Gene 292,Gene 293,Gene 294,Gene 295,Gene 296,Gene 297,Gene 298,Gene 299,Gene 300,Gene 301,Gene 302,Gene 303,Gene 304,Gene 305,Gene 306,Gene 307,Gene 308,Gene 309,Gene 310,Gene 311,Gene 312,Gene 313,Gene 314,Gene 315,Gene 316,Gene 317,Gene 318,Gene 319,Gene 320,Gene 321,Gene 322,Gene 323,Gene 324,Gene 325,Gene 326,Gene 327,Gene 328,Gene 329,Gene 330,Gene 331,Gene 332,Gene 333,Gene 334,Gene 335,Gene 336,Gene 337,Gene 338,Gene 339,Gene 340,Gene 341,Gene 342,Gene 343,Gene 344,Gene 345,Gene 346,Gene 347,Gene 348,Gene 349,Gene 350,Gene 351,Gene 352,Gene 353,Gene 354,Gene 355,Gene 356,Gene 357,Gene 358,Gene 359,Gene 360,Gene 361,Gene 362,Gene 363,Gene 364,Gene 365,Gene 366,Gene 367,Gene 368,Gene 369,Gene 370,Gene 371,Gene 372,Gene 373,Gene 374,Gene 375,Gene 376,Gene 377,Gene 378,Gene 379,Gene 380,Gene 381,Gene 382,Gene 383,Gene 384,Gene 385,Gene 386,Gene 387,Gene 388,Gene 389,Gene 390,Gene 391,Gene 392,Gene 393,Gene 394,Gene 395,Gene 396,Gene 397,Gene 398,Gene 399,Gene 400,Gene 401,Gene 402,Gene 403,Gene 404,Gene 405,Gene 406,Gene 407,Gene 408,Gene 409,Gene 410,Gene 411,Gene 412,Gene 413,Gene 414,Gene 415,Gene 416,Gene 417,Gene 418,Gene 419,Gene 420,Gene 421,Gene 422,Gene 423,Gene 424,Gene 425,Gene 426,Gene 427,Gene 428,Gene 429,Gene 430,Gene 431,Gene 432,Gene 433,Gene 434,Gene 435,Gene 436,Gene 437,Gene 438,Gene 439,Gene 440,Gene 441,Gene 442,Gene 443,Gene 444,Gene 445,Gene 446,Gene 447,Gene 448,Gene 449,Gene 450,Gene 451,Gene 452,Gene 453,Gene 454,Gene 455,Gene 456,G

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Columns: 6034 entries, Y to Gene 6033
dtypes: float64(6033), int64(1)
memory usage: 4.7 MB


In [4]:
df.describe()

Y      Gene 1      Gene 2      Gene 3      Gene 4      Gene 5      Gene 6      Gene 7      Gene 8      Gene 9     Gene 10     Gene 11     Gene 12     Gene 13     Gene 14     Gene 15     Gene 16     Gene 17     Gene 18     Gene 19     Gene 20     Gene 21     Gene 22     Gene 23     Gene 24     Gene 25     Gene 26     Gene 27     Gene 28     Gene 29     Gene 30     Gene 31     Gene 32     Gene 33     Gene 34     Gene 35     Gene 36     Gene 37     Gene 38     Gene 39     Gene 40     Gene 41     Gene 42     Gene 43     Gene 44     Gene 45     Gene 46     Gene 47     Gene 48     Gene 49     Gene 50     Gene 51     Gene 52     Gene 53     Gene 54     Gene 55     Gene 56     Gene 57     Gene 58     Gene 59     Gene 60     Gene 61     Gene 62     Gene 63     Gene 64     Gene 65     Gene 66     Gene 67     Gene 68     Gene 69     Gene 70     Gene 71     Gene 72     Gene 73     Gene 74     Gene 75     Gene 76     Gene 77     Gene 78     Gene 79     Gene 80     Gene 81  \
count  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000  102.000000   
mean     0.509804   -0.623086   -0.665809   -0.586693   -0.895592   -0.438002   -0.152997   -0.593764    4.361345   -0.553667   -0.951837    0.210988   -0.655313   -0.942147    1.056787    1.048407    2.420894    1.385449    1.984182    3.279895   -0.804082   -0.663843   -0.547087    0.982289    0.420177    0.589138   -0.654416   -0.763602   -0.177263   -0.354601   -0.733137    1.019639   -0.853698   -0.387593   -0.723554    0.333692   -0.870300    2.302401   -0.923628   -0.615913   -0.573766   -0.665838   -0.520763   -0.880360    0.206195   -0.512913   -0.927772   -0.818706   -0.693632   -0.077625   -0.674515   -0.792538   -0.596294   -0.943320    2.282409    0.475842   -0.598812    1.027012   -0.725107    0.342511   -0.816244   -0.278585   -0.498915   -0.177066   -0.528816   -0.262572   -0.948803   -0.852756   -0.833931   -0.218727   -0.653240    1.429048   -0.771873   -0.073120    0.287660    0.001412   -0.551145   -0.686902   -0.494723   -0.368184    0.538564    0.488985   
std      0.502373    0.409905    0.378207    0.497839    0.190989    0.597182    0.789601    0.414790    0.410420    0.399223    0.181319    0.606445    0.469975    0.243418    0.730810    1.528193    0.703898    0.712412    1.423613    0.860988    0.314300    0.455784    0.569803    0.607126    0.735268    0.669660    0.401457    0.292917    0.518869    0.560968    0.427445    0.629158    0.282037    0.527495    0.362698    0.837500    0.241137    1.421855    0.202965    0.667866    0.430846    0.352903    0.511037    0.225416    0.577369    0.537635    0.211668    0.339570    0.372069    0.664546    0.484781    0.311698    0.368384    0.191362    1.739226    0.716093    0.408567    0.581369    0.322546    1.304549    0.285212    0.568969    0.569240    0.597381    0.470440    0.765572    0.203211    0.264404    0.293816    0.563874    0.391422    0.642131    0.350769    0.536137    0.673359    0.957940    0.498118    0.460433    0.428367    0.638959    0.628258    0.855911   
min      0.000000   -1.130098   -1.206937   

`Y` here is the response variable (`Y` = 1 is cancer and `Y` = 0 is no cancer) and the 6033 predictor variables are gene expressions.Each row represents a person.

In [9]:
#Tarin test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( df.drop('Y',axis = 1), df['Y'], test_size=0.33, random_state=42)

In [11]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 28 to 51
Columns: 6033 entries, Gene 1 to Gene 6033
dtypes: float64(6033)
memory usage: 3.1 MB


In [14]:
y_train.shape

(68L,)

In [16]:
#Normalization
X_train_n = (X_train - X_train.min(axis=0))/(X_train.max(axis=0) - X_train.min(axis=0))
X_test_n = (X_test - X_train.min(axis=0))/(X_train.max(axis=0) - X_train.min(axis=0))

#### Using one single predictor to fit a Logistic Regression Model  
Using Gene 10

In [51]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(C=1000000, fit_intercept=True) #The C value here is a way to prevent regularization
X_train_gene10 = X_train["Gene 10"]
X_test_gene10 = X_test["Gene 10"]
log_reg.fit(X_train_gene10.reshape(-1,1), y_train)
y_predict_train_log = log_reg.predict(X_train_gene10.reshape(-1,1))
y_predict_test_log = log_reg.predict(X_test_gene10.reshape(-1,1))
print "Training Accuracy for Logistic Regression: ", accuracy_score(y_train, y_predict_train_log)
print "Test Accuracy for Logistic Regression: ", accuracy_score(y_test, y_predict_test_log)

Training Accuracy for Logistic Regression:  0.632352941176
Test Accuracy for Logistic Regression:  0.529411764706


In [52]:
log_reg.intercept_

array([ 1.18325051])

In [56]:
log_reg.coef_

array([[ 1.05701323]])

Calculating the probability of cancer in our training set  without using predict_proba() 

$$ p = \cfrac{e^\eta}{1+e^\eta} =  \cfrac{1}{1+e^{-\eta}}  $$

Where, $\eta = \alpha + \beta_1 X_1 + ... + \beta_n X_n$

In [70]:
eta = np.array(log_reg.coef_[0]*X_train['Gene 10']) + log_reg.intercept_
proba_man = 1/(1+np.exp(-eta))
proba_man 

array([ 0.50381999,  0.51050166,  0.50892305,  0.57011726,  0.51799324,
        0.55112321,  0.48618191,  0.47689209,  0.55040545,  0.50797849,
        0.54348604,  0.55229786,  0.517428  ,  0.59299254,  0.54397856,
        0.54546731,  0.57365142,  0.5818994 ,  0.49445238,  0.56384547,
        0.49944368,  0.48426111,  0.49156325,  0.49276196,  0.48464057,
        0.48457534,  0.51664639,  0.51728775,  0.56089594,  0.51605971,
        0.57962202,  0.57589114,  0.56106776,  0.49718061,  0.5682964 ,
        0.54893586,  0.50156691,  0.58797101,  0.57581275,  0.81265717,
        0.53643663,  0.57573595,  0.5813187 ,  0.59186358,  0.5578684 ,
        0.56464905,  0.58448636,  0.51186126,  0.48185011,  0.56748745,
        0.55013659,  0.57075723,  0.57436776,  0.52712799,  0.49045651,
        0.49077254,  0.51407176,  0.5818815 ,  0.61157109,  0.56121995,
        0.56226807,  0.54644301,  0.54277127,  0.55721174,  0.54834102,
        0.49367883,  0.59992973,  0.57286184])

In [71]:
proba_sklearn = log_reg.predict_proba(X_train_gene10.reshape(-1,1))[:,1]
proba_sklearn

array([ 0.50381999,  0.51050166,  0.50892305,  0.57011726,  0.51799324,
        0.55112321,  0.48618191,  0.47689209,  0.55040545,  0.50797849,
        0.54348604,  0.55229786,  0.517428  ,  0.59299254,  0.54397856,
        0.54546731,  0.57365142,  0.5818994 ,  0.49445238,  0.56384547,
        0.49944368,  0.48426111,  0.49156325,  0.49276196,  0.48464057,
        0.48457534,  0.51664639,  0.51728775,  0.56089594,  0.51605971,
        0.57962202,  0.57589114,  0.56106776,  0.49718061,  0.5682964 ,
        0.54893586,  0.50156691,  0.58797101,  0.57581275,  0.81265717,
        0.53643663,  0.57573595,  0.5813187 ,  0.59186358,  0.5578684 ,
        0.56464905,  0.58448636,  0.51186126,  0.48185011,  0.56748745,
        0.55013659,  0.57075723,  0.57436776,  0.52712799,  0.49045651,
        0.49077254,  0.51407176,  0.5818815 ,  0.61157109,  0.56121995,
        0.56226807,  0.54644301,  0.54277127,  0.55721174,  0.54834102,
        0.49367883,  0.59992973,  0.57286184])

In [76]:
(proba_sklearn == proba_man).sum()

68

In [77]:
np.array_equal(proba_sklearn,proba_man)

True

#### Using all predictor variables in a logistic regression

In [79]:
log_all = LogisticRegression(C=100000, fit_intercept=True)
log_all.fit(X_train_n, y_train)
y_pred_log_all_train = log_all.predict(X_train_n)
y_pred_log_all_test = log_all.predict(X_test_n)
print("Training accuracy for Logistic Regression: ", accuracy_score(y_train, y_pred_log_all_train))
print("Test accuracy for Logistic Regression: ", accuracy_score(y_test, y_pred_log_all_test))

('Training accuracy for Logistic Regression: ', 1.0)
('Test accuracy for Logistic Regression: ', 0.94117647058823528)


In [81]:
log_all.coef_

array([[-0.02457213, -0.04006059, -0.09820024, ...,  0.03252062,
        -0.03102337, -0.11788837]])

In [82]:
log_all.intercept_

array([ 0.00370287])

In [83]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred_log_all_test)

array([[18,  1],
       [ 1, 14]], dtype=int64)

In [84]:
print(metrics.classification_report(y_test,y_pred_log_all_test))

             precision    recall  f1-score   support

          0       0.95      0.95      0.95        19
          1       0.93      0.93      0.93        15

avg / total       0.94      0.94      0.94        34



#### Incorporating Bootstrapping in a Logistic Regression Model

The main idea behind bootstrapping is to resample our data many times. For each time that we resample our data, we will fit the Logistic Regression model. Then for each resampling, we can find an associated confidence interval. Notice how we have many more predictors than observations. Intuitively, that creates a problem in that we do not have enough observations to "represent" each predictor varaible. Hence, the resampling gives us a way to approximate our standard errors. The goal here is to find confidence intervals for our logistic regression model using ALL the predictors. You may ask, why don't we just fit a logistic regression, then look at the reported standard errors and confidence intervals? The answer is that with more predictors than observations, the reported confidence intervals will look like NaN.   


#### a bootstrap analysis and estimate just how many of our coefficients are significant

In [96]:
NOB = 100 # Number of iterations
boot_coefs = np.zeros((X_train_n.shape[1],NOB)) # Create empty storage array for later use
for i in range(NOB):
    # Sampling WITH replacement the indices of a resampled dataset
    sample_index = np.random.choice(range(len(y_train)), size=len(y_train), replace=True)
    X_train_samples = X_train_n.values[sample_index]
    y_train_samples = y_train.values[sample_index]  
    log_all_boot = LogisticRegression(C=100000, fit_intercept=True)
    log_all_boot.fit(X_train_samples, y_train_samples)
    boot_coefs[:,i] = log_all_boot.coef_
boot_coefs.shape

(6033L, 100L)

If we wanted to find the 95 percent confidence interval, we can use the np.percentile function to find the datapoint that corresponds to the upper and lower bounds. As an example, we have the upper datapoint for the 95% confidence interval. (Why is it 97.5? Think of an interval from 0 to 100, and you want to shade in 95% of that interval, while keeping the length to the left and right of the shading the same. Where does the shading end and begin?)

In [98]:
ci_upper = np.percentile(boot_coefs, 97.5, axis=1)
ci_lower = np.percentile(boot_coefs, 2.5, axis=1)

In [102]:
# ct significant predictors
sig_b_ct = 0
# if ci contains 0, then insignificant
for i in range(len(ci_upper)):
    if ci_upper[i]<0 or ci_lower[i]>0:
        sig_b_ct += 1
print("Significant coefficents at 5pct level = %i / %i" % (sig_b_ct, X_train_n.shape[1]))

Significant coefficents at 5pct level = 1549 / 6033
